## Fusion des indices avec la base EHCVM niveau département

In [16]:
# ======================================================
# fusion_ehcvm_indices.py
# Vérification + préparation fusion EHCVM – Indices spectraux
# ======================================================

import os
import pandas as pd
import pyreadstat
import unicodedata

# ======================================================
# 1️⃣ Fonctions utilitaires
# ======================================================
def normaliser_nom(nom):
    """
    Normalise les noms de départements :
    - minuscules
    - suppression accents
    - suppression espaces multiples
    """
    if pd.isna(nom):
        return None
    nom = str(nom).strip().lower()
    nom = unicodedata.normalize("NFD", nom)
    nom = "".join(c for c in nom if unicodedata.category(c) != "Mn")
    nom = nom.replace("-", " ")
    nom = " ".join(nom.split())
    return nom

# ======================================================
# 2️⃣ Chemins
# ======================================================
BASE_DIR = os.getcwd()  # tp/scripts
DATA_DIR = os.path.join(BASE_DIR, "..", "data")
OUTPUT_DIR = os.path.join(BASE_DIR, "..", "outputs")

# Base EHCVM
path_ehcvm = os.path.join(DATA_DIR, "s00_me_MLI2021.dta")

# Indices spectraux
indices_paths = {
    "NDVI": os.path.join(OUTPUT_DIR, "NDVI", "NDVI_departements.csv"),
    "NDWI": os.path.join(OUTPUT_DIR, "NDWI", "NDWI_departements.csv"),
    "NDMI": os.path.join(OUTPUT_DIR, "NDMI", "NDMI_departements.csv"),
    "NDBI": os.path.join(OUTPUT_DIR, "NDBI", "NDBI_departements.csv"),
    "BSI":  os.path.join(OUTPUT_DIR, "BSI",  "BSI_departements.csv")
}

# ======================================================
# 3️⃣ Chargement base EHCVM
# ======================================================
ehcvm, meta = pyreadstat.read_dta(path_ehcvm)
print("✔ Base EHCVM chargée")
print(f"Observations : {ehcvm.shape[0]}, Variables : {ehcvm.shape[1]}")

# ======================================================
# 4️⃣ Table de correspondance codes ↔ noms départements
# ======================================================
if "s00q02" not in ehcvm.columns:
    raise ValueError("❌ Variable département s00q02 absente")

# Extraction des labels Stata
dept_labels = meta.value_labels["s00q02"]
correspondance_depts = (
    pd.DataFrame.from_dict(dept_labels, orient="index", columns=["nom_dept"])
    .reset_index()
    .rename(columns={"index": "code_dept"})
)

# Fusion pour ajouter les noms dans EHCVM
ehcvm = ehcvm.merge(
    correspondance_depts,
    left_on="s00q02",
    right_on="code_dept",
    how="left"
)

# Normalisation
ehcvm["dept_norm"] = ehcvm["nom_dept"].apply(normaliser_nom)
ehcvm_depts = set(ehcvm["dept_norm"].dropna().unique())
print("\n📌 EHCVM")
print(f"Nombre de départements : {len(ehcvm_depts)}")

# ======================================================
# 5️⃣ Chargement indices spectraux
# ======================================================
indices_tables = {}

for indice, path in indices_paths.items():
    if not os.path.exists(path):
        raise FileNotFoundError(f"❌ Fichier manquant : {path}")
    
    df = pd.read_csv(path)
    
    if "shapeName" not in df.columns:
        raise ValueError(f"❌ Colonne shapeName absente dans {indice}")
    
    # Normalisation des noms pour comparaison
    df["dept_norm"] = df["shapeName"].apply(normaliser_nom)
    
    indices_tables[indice] = df
    print(f"\n📊 {indice}")
    print(f"Colonnes : {list(df.columns)}, Départements : {df.shape[0]}")

# ======================================================
# 6️⃣ Comparaison EHCVM ↔ indices
# ======================================================
print("\n================ COMPARAISON DES DÉPARTEMENTS ================")

for indice, df in indices_tables.items():
    indices_depts = set(df["dept_norm"].dropna().unique())
    communs = ehcvm_depts.intersection(indices_depts)
    absents_ehcvm = indices_depts - ehcvm_depts
    absents_indices = ehcvm_depts - indices_depts

    print(f"\n🔎 {indice}")
    print(f"Départements communs : {len(communs)}")
    if absents_ehcvm:
        print("⚠ Présents dans indices mais absents EHCVM :", sorted(absents_ehcvm))
    if absents_indices:
        print("⚠ Présents dans EHCVM mais absents indices :", sorted(absents_indices))



✔ Base EHCVM chargée
Observations : 6143, Variables : 25

📌 EHCVM
Nombre de départements : 56

📊 NDVI
Colonnes : ['shapeName', 'NDVI', 'dept_norm'], Départements : 50

📊 NDWI
Colonnes : ['shapeName', 'NDWI', 'dept_norm'], Départements : 50

📊 NDMI
Colonnes : ['shapeName', 'NDMI', 'dept_norm'], Départements : 50

📊 NDBI
Colonnes : ['shapeName', 'NDBI', 'dept_norm'], Départements : 50

📊 BSI
Colonnes : ['shapeName', 'BSI', 'dept_norm'], Départements : 50

================ COMPARAISON DES DÉPARTEMENTS ================

🔎 NDVI
Départements communs : 48
⚠ Présents dans indices mais absents EHCVM : ['bamako', 'gourma rharous']
⚠ Présents dans EHCVM mais absents indices : ['achouratt', 'alourche', 'arawane', 'bouzbeha', 'district de bamako', 'foum elba', 'gourma rha', 'taoudenit']

🔎 NDWI
Départements communs : 48
⚠ Présents dans indices mais absents EHCVM : ['bamako', 'gourma rharous']
⚠ Présents dans EHCVM mais absents indices : ['achouratt', 'alourche', 'arawane', 'bouzbeha', 'district de 

In [22]:
# ======================================================
# 7️⃣ Corrections manuelles des noms particuliers
# ======================================================
# Certains départements peuvent avoir des noms légèrement différents
corrections = {
    "bamako": "district de bamako",
    "gourma rharous": "gourma rha"
}

for df in indices_tables.values():
    df["dept_norm"] = df["dept_norm"].replace(corrections)

# ======================================================
# 8️⃣ Fusion finale EHCVM × indices
# ======================================================
ehcvm_fusion = ehcvm.copy()

for indice, df in indices_tables.items():
    ehcvm_fusion = ehcvm_fusion.merge(
        df[["dept_norm", indice]],
        on="dept_norm",
        how="left"
    )

# ======================================================
# 9️⃣ Sauvegarde de la base fusionnée
# ======================================================
output_file = os.path.join(OUTPUT_DIR, "EHCVM_indices_fusion.csv")
ehcvm_fusion.to_csv(output_file, index=False)
print(f"\n✅ Base fusionnée enregistrée : {output_file}")



✅ Base fusionnée enregistrée : C:\Users\HP\Desktop\ISEP3\statistique exploratoire spatiale\TP6_groupe3\scripts\..\outputs\EHCVM_indices_fusion.csv
